In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
import requests
import utils
from bs4 import BeautifulSoup
from urllib.parse import urlparse
import tld

In [6]:
def is_valid_url(url):
    try:
        urlparse(url)
        return True
    except Exception:
        return False

In [28]:
def get_external_links(html, tld_filter):
    soup = BeautifulSoup(html, features="html.parser")
    a_s = soup.find_all("a")
    return [
        a.attrs["href"]
        for a in a_s
        if "href" in a.attrs
        and "http" in a.attrs["href"]
        and is_valid_url(a.attrs["href"])
        and tld.get_tld(a.attrs["href"]) == tld_filter
    ]

In [29]:
def get_random_urls(count=2, lang="nl"):
    links = []

    random_pages = {
        "nl": "https://nl.wikipedia.org/wiki/Speciaal:Willekeurig",
        "de": "https://de.wikipedia.org/wiki/Spezial:Zuf%C3%A4llige_Seite",
    }

    for i in range(count):
        print(i, end='')
        resp = requests.get(random_pages[lang])
        links += get_external_links(resp.content, lang)

    return links

In [30]:
links = get_random_urls(count=50); links

012345678910111213141516171819202122232425262728293031323334353637383940414243444546474849

['http://hetperfecteplaatje.rtl.nl',
 'http://hetperfecteplaatje.rtl.nl/',
 'https://www.ad.nl/show/humberto-tan-wint-het-perfecte-plaatje~a150001c/',
 'https://www.telegraaf.nl/entertainment/1424891/estelle-cruijff-winnares-het-perfecte-plaatje',
 'https://www.televizier.nl/amusement/het-perfecte-plaatje-krijgt-derde-seizoen',
 'https://www.ad.nl/show/eerste-drie-bn-ers-het-perfecte-plaatje-bekend~a3def931/',
 'https://www.ad.nl/show/patty-brard-uitgeroepen-tot-beste-bekende-fotograaf~a19d6d59/',
 'https://www.telegraaf.nl/entertainment/3645446/dit-zijn-de-deelnemers-van-het-perfecte-plaatje',
 'https://www.nu.nl/media/6018876/winnaar-van-fotografiewedstrijd-het-perfecte-plaatje-bekend.html',
 'https://www.ad.nl/show/kijkcijferrecord-ruim-1-6-miljoen-kijkers-voor-finale-het-perfecte-plaatje-2020~ad78817f/',
 'http://www.wikiportret.nl/',
 'https://www.korfbal.nl/korfballeague/nieuws/frank-mostard-van-blauw-wit-naar-top',
 'https://www.nhnieuws.nl/nieuws/225387/frank-mostard-en-peter-s